In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import warnings
warnings.filterwarnings("ignore")
import yfinance as yf
from tqdm import tqdm
import seaborn as sns
from scipy.optimize import minimize
from Backtest import *

ModuleNotFoundError: No module named 'tqdm'